In [31]:
import pickle

In [32]:
# add path
import sys
sys.path.append('..')

In [33]:
from reasoners.visualization import TreeLog

In [34]:
node_translator = lambda node: {"blocks_state": node.state.blocks_state}

In [35]:
def get_reward(node):
    if hasattr(node, "reward_details"):
        return node.reward_details
    else:
        return node.fast_reward_details

In [44]:
log = TreeLog.from_MCTSResults("/home/shibo/LLM-search/logs/bw_MCTS/07202023-033526/algo_output/2.pkl", lambda node: {"blocks_state": node.state.blocks_state if node.state else None}, lambda node: {"Q": node.Q, **get_reward(node)})

In [45]:
nodes, edges = log.get_tree(9).visualize()